In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/home/tdelatte/new-projects/ethereum-analytics/key/ethereum-analytics-309308-6c01508bc0b8.json"

In [3]:
from google.cloud import bigquery
client = bigquery.Client()

In [9]:
# All data
QUERY = ''' SELECT contracts.address, COUNT(1) AS tx_count
            FROM `bigquery-public-data.crypto_ethereum.contracts` AS contracts
            JOIN `bigquery-public-data.crypto_ethereum.transactions` AS transactions ON (transactions.to_address = contracts.address)
            GROUP BY contracts.address
            ORDER BY tx_count DESC
            LIMIT 10 '''

In [7]:
QUERY_2 = '''SELECT *
            FROM `bigquery-public-data.crypto_ethereum.contracts` AS contracts
            LIMIT 10 '''

In [10]:
def load_data_from_bigquery(QUERY):
    
    query_job = client.query(QUERY) # API request
    df = query_job.to_dataframe()
    
    return df

In [12]:
df_query = load_data_from_bigquery(QUERY)

In [13]:
df_query.head(10)

,address,tx_count
0,0xdac17f958d2ee523a2206206994597c13d831ec7,91101394
1,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,30478160
2,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,11530548
3,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,11530428
4,0x000000000000006f6502b7f2bbac8c30a3f67e9a,11188080
5,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,9779959
6,0x174bfa6600bf90c885c7c01c7031389ed1461ab9,9088226
7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,8038532
8,0x06012c8cf97bead5deae237070f9587f8e7a266d,4826257
9,0xd1ceeeeee83f8bcf3bedad437202b6154e9f5405,4225204


In [14]:
df_query["tx_count"].max()

91101394

In [9]:
df_2 = load_data_from_bigquery(QUERY_2)

In [10]:
df_2.head()

,address,bytecode,function_sighashes,is_erc20,is_erc721,block_timestamp,block_number,block_hash
0,0xa96c524d46cff3ee39ab04ce22beee06329764ad,0x60606040526000357c01000000000000000000000000...,"[0x7c67a3e2, 0x91a89712, 0xfdcdf85e, 0xffffffff]",False,False,2017-03-17 11:01:51+00:00,3367781,0x1ecdf4c6528ad09783e72cc85049e7f5e7f3a931bef8...
1,0x697ed9a0ef0d1481ecb36fc123e85da40f1bc8d5,0x606060405236156101d25763ffffffff60e060020a60...,"[0x02571be3, 0x02927d20, 0x0610e037, 0x085a470...",False,False,2017-03-17 11:25:32+00:00,3367889,0xe7b490eab2b29fb3ada14004ea055bc446ef2487ea17...
2,0x4c5e646669cc72161fb5c73c31d70354b192dc29,0x606060405236156100ab576000357c01000000000000...,"[0x2e1a7d4d, 0x38d52e0f, 0x4b750334, 0x4ca50f5...",False,False,2017-03-17 15:54:17+00:00,3369032,0x4d67ade0e0c4ee2241dd996a46470ee4b340ae1ebef0...
3,0x89145f218a049963af6575ce8dfbe503fec73037,0x606060405236156100da5760e060020a600035046317...,"[0x173825d9, 0x2f54bf6e, 0x4123cb6b, 0x5237509...",False,False,2017-03-17 18:27:59+00:00,3369671,0xf8ff9955c0e8c197704ffde43877046cbae6f93397cb...
4,0x5bb4ef88cf97d29526862494077563a2d97d7443,0x606060405263ffffffff60e060020a6000350416630f...,"[0x0fe28774, 0x8da5cb5b, 0x9142db42, 0xd6bf08b...",False,False,2017-03-17 11:33:48+00:00,3367921,0xea74c09ab3614bd7be25cafa25723b619b8adb595a25...


In [11]:
df_2.columns

Index(['address', 'bytecode', 'function_sighashes', 'is_erc20', 'is_erc721',
       'block_timestamp', 'block_number', 'block_hash'],
      dtype='object')

In [14]:
QUERY_3 = '''SELECT *
from `bigquery-public-data.crypto_ethereum.balances`
order by eth_balance desc
limit 100 '''

In [15]:
df_3 = load_data_from_bigquery(QUERY_3)

In [16]:
df_3.head()

,address,eth_balance
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,6909232283529961722637579.000000000
1,0x00000000219ab540356cbb839cbe05303d7705fa,3607682000069000000000069.000000000
2,0xbe0eb53f46cd790cd13851d5eff43d12404d33e8,2296896749617084491021288.000000000
3,0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5,1767412610958686264700881.000000000
4,0xc61b9bb3a7a0767e3179713f3a5c7a9aedce193c,1700010750604813220000000.000000000


In [ ]:
def feature_extraction_training_instances(num_instances):
    query = f"""


        WITH
          -- current ethereum balance for 1000 non-zero balance addresses
          ethereum_balance AS (
          SELECT
            address AS ethereum_address,
            (eth_balance / POWER(10, 18)) AS eth_balance
          FROM
            `bigquery-public-data.crypto_ethereum.balances`
          WHERE
            (eth_balance / POWER(10, 18)) > 1
          LIMIT
            {num_instances}),
          top_tokens AS (
          SELECT
            token_address,
            COUNT(1) AS transfer_count
          FROM
            `bigquery-public-data.ethereum_blockchain.token_transfers` AS token_transfers
          GROUP BY
            token_address
          ORDER BY
            transfer_count DESC
          LIMIT
            1000 ),
          token_balances AS (
          WITH
            double_entry_book AS (
            SELECT
              token_address,
              to_address AS ethereum_address,
              CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers`
            UNION ALL
            SELECT
              token_address,
              from_address AS ethereum_address,
              -CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers` )
          SELECT
            a.ethereum_address,
            b.token_address,
            SUM(value) AS balance,
            COUNT(DISTINCT transaction_hash) as unique_transfers
          FROM
            ethereum_balance a
          JOIN
            double_entry_book b
          ON
            a.ethereum_address = b.ethereum_address
          JOIN
            top_tokens c
          ON
            c.token_address = b.token_address
          WHERE
            a.ethereum_address != '0x0000000000000000000000000000000000000000'
          GROUP BY
            1,
            2
          HAVING
            balance > 0 )
        SELECT
          ethereum_address,
          MAX(eth_balance) AS ether_balance,
          COUNT(DISTINCT token_address) AS unique_tokens,
          MAX(unique_transfers) as unique_transfers
        FROM
          ethereum_balance a
        JOIN
          token_balances b
        USING
          (ethereum_address)
        GROUP BY
          1

    """

    # Pandas GBQ, wrapper
    # module provides a wrapper for Google’s BigQuery analytics web service

    address_features = pandas_gbq.read_gbq(query, credentials=credentials)
    return address_features

train_x_raw = feature_extraction_training_instances(num_instances=10000)
train_x_raw.head()

In [5]:
QUERY_5 = f"""


        WITH
          ethereum_balance AS (
          SELECT
            address AS ethereum_address,
            (eth_balance / POWER(10, 18)) AS eth_balance
          FROM
            `bigquery-public-data.crypto_ethereum.balances`
          WHERE
            (eth_balance / POWER(10, 18)) > 1000
          LIMIT
            1000),
          
          top_tokens AS (
          SELECT
            token_address,
            COUNT(1) AS transfer_count
          FROM
            `bigquery-public-data.ethereum_blockchain.token_transfers` AS token_transfers
          GROUP BY
            token_address
          ORDER BY
            transfer_count DESC
          LIMIT
            1000 ),
         
         token_balances AS (
          WITH
            double_entry_book AS (
            SELECT
              token_address,
              to_address AS ethereum_address,
              CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers`
            UNION ALL
            SELECT
              token_address,
              from_address AS ethereum_address,
              -CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers` )
          SELECT
            a.ethereum_address,
            b.token_address,
            SUM(value) AS balance,
            COUNT(DISTINCT transaction_hash) as unique_transfers
          FROM
            ethereum_balance a
          JOIN
            double_entry_book b
          ON
            a.ethereum_address = b.ethereum_address
          JOIN
            top_tokens c
          ON
            c.token_address = b.token_address
          WHERE
            a.ethereum_address != '0x0000000000000000000000000000000000000000'
          GROUP BY
            1,
            2
          HAVING
            balance > 0 )
            
            
        SELECT
          ethereum_address,
          MAX(eth_balance) AS ether_balance,
          COUNT(DISTINCT token_address) AS unique_tokens,
          MAX(unique_transfers) AS unique_transfers
        FROM
          ethereum_balance a
        JOIN
          token_balances b
        USING
          (ethereum_address)
        GROUP BY
          1

    """

In [6]:
df_5 = load_data_from_bigquery(QUERY_5)

In [7]:
df_5.head()

,ethereum_address,ether_balance,unique_tokens,unique_transfers
0,0x725120bc6b16dcd84b29e1f706de6eecb000faec,1021.079985,16,2
1,0x6b18a4aa74dece98107eff906fc1c9da0b4834a4,1728.997624,8,1
2,0x56205c6547f4132384b083679dd8fb62eedb5da0,2805.068624,23,2
3,0xae14bf447b9e333c3c4dea73f85ed1ec21de4df4,6575.388154,22,2
4,0x05cdb1526f6e224e02919a4c018d9784ea25eb3d,5149.858570,1,1


In [8]:
df_5["unique_transfers"].max()

1596

In [38]:
QUERY_10 = '''
SELECT block_number, gas_price 
FROM bigquery-public-data.crypto_ethereum.transactions
ORDER BY block_number
LIMIT 100'''

In [39]:
df_10 = load_data_from_bigquery(QUERY_10)

In [40]:
df_10.head()

,block_number,gas_price
0,46147,50000000000000
1,46169,909808707606
2,46170,500000000000
3,46194,1000000000000
4,46205,500000000000
